# Neural Network Image Processing for the JPL Space Design Competition

## or: How I Learned to Stop Worrying and Love the Net

---

# Problem:

**Can we use neural networks to identify "safe" landing sites on a planetary surface?**

# Provided Information

* 4 Training sets

![](heights_colormap.png)
![](heights_3d.png)